In [ ]:
%matplotlib inline
import market
import metrics

from decimal import Decimal
import pandas
import seaborn

In [ ]:
m = market.Returns_US_1871()
years = 18
start_year = 1970
last_year = 2018-years

def run(stock_pct, final=0):
    swrs = pandas.Series()
    for _ in range(last_year - start_year + 2):
        returns = []
        for one_year in zip(range(years), m.iter_from(start_year + _)):
            annual_returns = one_year[1]
            stocks = ((1+annual_returns.stocks) / (1+annual_returns.inflation)) - 1
            bonds = ((1+annual_returns.bonds) / (1+annual_returns.inflation)) - 1
            returns.append((stocks * stock_pct) + (bonds * (1-stock_pct)))
        swr = float(metrics.pwa(100, final, returns))
        swrs.loc[start_year+_] = swr
    return swrs

six_swr = run(Decimal('.6'))
six_pwr = run(Decimal('.6'), final=100)

g = seaborn.relplot(
    data=pandas.DataFrame({'SWR' : six_swr, 'PWR' : six_pwr}),
    kind='line',
    aspect=2,
    )
g.fig.autofmt_xdate()
g.despine(left=True, bottom=True, offset=20)
g.fig.suptitle('SWR vs PWR (60/40 portfolio)')

In [ ]:
six_swr.sort_values().head()

In [ ]:
six_pwr.sort_values().head()

In [ ]:
import scipy.stats
import simulate
import harvesting
import withdrawal

In [ ]:
scipy.stats.percentileofscore(six_pwr, 2.5)

In [ ]:
portfolio = (600000,400000)    
s = simulate.withdrawals(m.iter_from(1971),
                             withdraw=lambda p, s: withdrawal.ConstantDollar(p, s, rate=Decimal('0.025')),
                             years=18,
                             portfolio=portfolio,
                             harvesting=harvesting.make_rebalancer(0.6))
df = pandas.DataFrame({
    '60_40' : [x.portfolio_post.value_r for x in s],
})
g = seaborn.relplot(
        data=df,
        aspect=2,
    )
g.fig.autofmt_xdate()
g.despine(left=True, bottom=True, offset=20)

In [ ]:
df['60_40'].sort_values().head()